In [ ]:
!pip install Konlpy > /dev/null

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep = '\t')
test_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', sep = '\t')

In [ ]:
print(train_df.shape, test_df.shape)
train_df.head()

(150000, 3) (50000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


## 1.데이터 전처리

In [8]:
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [9]:
train_df.dropna(how = 'any', inplace = True)
train_df.shape

(149995, 3)

In [ ]:
# 중복 데이터 확인
train_df.document.nunique()

146182

In [10]:
# 중복 데이터 제거
train_df.drop_duplicates(subset = ['document'], inplace = True)
train_df.shape

(146182, 3)

In [ ]:
# 긍부정 개수 확인
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- test 전처리


In [ ]:
test_df.isna().sum()

id          0
document    3
label       0
dtype: int64

In [3]:
test_df.dropna(how = 'any', inplace = True)
test_df.shape

(49997, 3)

In [4]:
test_df.document.nunique()

49157

In [5]:
test_df.drop_duplicates(subset = ['document'], inplace = True)

In [ ]:
test_df.shape

(49157, 3)

In [ ]:
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

## 2.Document 전처리

In [11]:
# 한글 이외 글자 공백으로
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ').str.strip()
train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [12]:
# ''만 남은 데이터 -> np.nan으로 대체 후 제거
train_df.document.replace('', np.nan, inplace = True)
train_df.document.isna().sum()

789

In [13]:
train_df.dropna(how = 'any', inplace = True)
train_df.shape

(145393, 3)

- test

In [6]:
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ').str.strip()
test_df.document.replace('', np.nan, inplace = True)
test_df.dropna(how = 'any', inplace = True)
test_df.shape

(48852, 3)

- 전처리 끝난 데이터 save

In [15]:
train_df.to_csv('naver_movie_train.tsv', sep = '\t', index=False)

In [7]:
test_df.to_csv('naver_movie_test.tsv', sep = '\t', index=False)

## 3.한글처리

In [ ]:
from konlpy.tag import Okt
okt = Okt()

In [ ]:
text = '어제 너는 나를 버렸어 나는 아무 말도 하지 못하고'
okt.morphs(text)

['어제', '너', '는', '나를', '버렸어', '나', '는', '아무', '말', '도', '하지', '못', '하고']

In [ ]:
okt.morphs(text, stem = True)

['어제', '너', '는', '나르다', '버리다', '나', '는', '아무', '말', '도', '하다', '못', '하고']

In [ ]:
okt.pos(text)

[('교도소', 'Noun'),
 ('이야기', 'Noun'),
 ('구먼', 'Noun'),
 ('솔직히', 'Adjective'),
 ('재미', 'Noun'),
 ('는', 'Josa'),
 ('없다', 'Adjective'),
 ('평점', 'Noun'),
 ('조정', 'Noun')]

In [ ]:
stopwords = ['이','가','의','은','들','는','좀','잘','걍',
             '과','도','를','으로','자','에','와','한','하다','을','ㅋ']

In [ ]:
' '.join([word for word in okt.morphs(text, stem = True) if word not in stopwords])

'어제 너 나르다 버리다 나 아무 말 못 하고'

In [ ]:
from tqdm import tqdm

X_train = []
for sentence in tqdm(train_df.document):
    morphs = okt.morphs(sentence, stem = True)
    tmp_str =  ' '.join([word for word in okt.morphs(text, stem = True) if word not in stopwords])
    X_train.append(tmp_str)

100%|██████████| 145393/145393 [08:05<00:00, 299.53it/s]


In [ ]:
X_test = []
for sentence in tqdm(test_df.document):
    morphs = okt.morphs(sentence, stem = True)
    tmp_str =  ' '.join([word for word in okt.morphs(text, stem = True) if word not in stopwords])
    X_test.append(tmp_str)

100%|██████████| 48852/48852 [02:40<00:00, 303.56it/s]


In [ ]:
y_train = train_df.label.values
y_test = test_df.label.values

## 4.Feature 변환 및 모델 학습
- Pipeline 
- TfidfVectorizer + LogisticRegression

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [ ]:
cvec = CountVectorizer()
lr = LogisticRegression(random_state = 2022)
pipline = Pipeline([('CVECT', cvec), ("LR", lr)])
%time pipline.fit(X_train, y_train)

CPU times: user 1.7 s, sys: 32 ms, total: 1.73 s
Wall time: 1.8 s


Pipeline(steps=[('CVECT', CountVectorizer()),
                ('LR', LogisticRegression(random_state=2022))])

In [ ]:
pipline.score(X_test, y_test)

0.4980758208466388

In [ ]:
# 실제 데이터 예측 
t1 = '방금 보고나왔는데 요 최근 한국영화중 최고.. 배우들 연기는 물론이고 스토리도 최고였음 얼른 이 문제 해결되길 바람... 대한민국 국민이면 이 영화 한번 쯤 보는게 좋을 듯'
t2 = '이것은 액션인가 로맨스인가. 지금까지 이런 영화는 많았다~. 비싼 배우님들 모시고 이게 뭐하는 짓이냐 !!!'

In [ ]:
# 전처리
import re
t1 = re.sub('[^가-힣]',' ',t1)
t2 = re.sub('^[가-힣]',' ', t2)

In [ ]:
pipline.predict([t1])

array([0])

In [ ]:
pipline.predict([t2])

array([0])

## 6.교차검증. 최적파라미터찾기

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
    'CVECT__ngram_range': [(1,1),(1,2)],
    'CVECT__max_df': [0.95, 0.98],
    'LR__C': [1, 5]
}

In [ ]:
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3, n_jobs=-1)
%time grid_pipe.fit(X_train, y_train)

In [ ]:
grid_pipe.best_params_

In [ ]:
grid_pipe.best_estimator_.score(X_test, y_test)